In [1]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "spark-3.5.3-bin-hadoop3"

In [2]:
import findspark
findspark.init()

In [3]:
import multiprocessing
print(multiprocessing.cpu_count())

8


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSV Loader") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext  # Get the existing SparkContext

25/03/20 15:59:55 WARN Utils: Your hostname, Duncans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.18.95.63 instead (on interface en0)
25/03/20 15:59:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 15:59:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/20 15:59:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
%%time
#removing header
reddit_rdd = sc.textFile("reddit_full_cleaned/reddit_full.csv")
header=reddit_rdd.first()
reddit_rdd=reddit_rdd.filter(lambda row : row!= header)

CPU times: user 2.5 ms, sys: 4.1 ms, total: 6.6 ms
Wall time: 307 ms


In [15]:
%%time
pos_words_en=set()
neg_words_en=set()
pos_words_es=set()
neg_words_es=set()
pos_words_fr=set()
neg_words_fr=set()
pos_words_tr=set()
neg_words_tr=set()
pos_words_it=set()
neg_words_it=set()

with open('sentiment_txts/positive_words_en.txt','r') as file:
    for line in file:
        pos_words_en.add(line.strip())
with open('sentiment_txts/negative_words_en.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_en.add(line.strip())
with open('sentiment_txts/positive_words_es.txt','r') as file:
    for line in file:
        pos_words_es.add(line.strip())
with open('sentiment_txts/negative_words_es.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_es.add(line.strip())
with open('sentiment_txts/positive_words_fr.txt','r') as file:
    for line in file:
        pos_words_fr.add(line.strip())
with open('sentiment_txts/negative_words_fr.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_fr.add(line.strip())
with open('sentiment_txts/positive_words_tr.txt','r') as file:
    for line in file:
        pos_words_tr.add(line.strip())
with open('sentiment_txts/negative_words_tr.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_tr.add(line.strip())
with open('sentiment_txts/positive_words_it.txt','r') as file:
    for line in file:
        pos_words_it.add(line.strip())
with open('sentiment_txts/negative_words_it.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_it.add(line.strip())

CPU times: user 10.9 ms, sys: 4.67 ms, total: 15.6 ms
Wall time: 17.1 ms


In [50]:
reddit_rdd.take(5)

['hi1vsag,When you scheduled your booster with CVS does it just give you the option of Vaccines: COVID-19 (Vaccine brand) or does it specifically say booster',
 'hi1vs7i,"Didn\'t stop prices there though. New Zealand and Canada grew at about the same rate through COVID.   I agree that non-resident ownership should be stopped, but it\'s also probably not going to change anything."',
 'hi1vs5n,I’m just waiting until after pregnancy to get vaccinated.  I have a special needs daughter due to having fever during pregnancy and I’m not willing to risk the vaccine fever right now. I have strong antibodies from Covid that I had before I got pregnant. (I keep taking blood tests to check) There’s a difference between anti vaccine and deciding to wait based on different factors',
 'hi1vs5v,"*The first duty of every Starfleet officer is to the truth. Whether it\'s scientific truth, or historical truth, or personal truth. It is the guiding principle upon which Starfleet is based. If you can\'t find 

In [16]:
import re
#input: line of rdd, output: list of (id,word) pairs for each cleaned word in text
def reddit_id_word_pair(line):
    id, text = line.split(",",1) 
    cleaned_text=re.sub(r'[^a-zA-Z\s]','',text.strip()).lower()
    return [(id,word) for word in re.split(r'\s+',cleaned_text)]

In [17]:
def reddit_word_to_sentiment(pair):
    id, word = pair
    score=0
    if word in pos_words_en:
        score+= 1
    if word in neg_words_en:
        score-=1
    return id, score

In [18]:
%%time
#MapReduce
reddit_id_word_pair_rdd=reddit_rdd.flatMap(reddit_id_word_pair)
reddit_sentiment_pair_rdd=reddit_id_word_pair_rdd.map(reddit_word_to_sentiment)
reddit_comment_scores=reddit_sentiment_pair_rdd.reduceByKey(lambda x,y: x+y)

CPU times: user 5.02 ms, sys: 6.2 ms, total: 11.2 ms
Wall time: 42 ms


In [19]:
%%time
#filtering positive, neutral, negative
reddit_positive_scores=reddit_comment_scores.filter(lambda x: x[1]>0)
reddit_neutral_scores=reddit_comment_scores.filter(lambda x: x[1]==0)
reddit_negative_scores=reddit_comment_scores.filter(lambda x: x[1]<0)

CPU times: user 48 μs, sys: 158 μs, total: 206 μs
Wall time: 217 μs


In [20]:
%%time
total=reddit_comment_scores.count()
print(f"total: {total}")
reddit_positive_percent=reddit_positive_scores.count()/total
reddit_neutral_percent=reddit_neutral_scores.count()/total
reddit_negative_percent=reddit_negative_scores.count()/total
print(f"pos: {reddit_positive_percent} neutral: {reddit_neutral_percent} neg: {reddit_negative_percent}")

total: 17777472


[Stage 17:===================================================>  (285 + 8) / 296]

pos: 0.32261915529947116 neutral: 0.2518384503713464 neg: 0.42554239432918245
CPU times: user 360 ms, sys: 244 ms, total: 604 ms
Wall time: 5min 35s


In [8]:
def twitter_id_word_pair(line):
    id, lang, text = line.split(",",2) 
    cleaned_text=re.sub(r'[^a-zA-Z\s]','',text.strip()).lower()
    return [(id,lang,word) for word in re.split(r'\s+',cleaned_text)]

def twitter_word_to_sentiment(pair):
    id, lang, word = pair
    score=0
    if lang == "en" or lang == 'und':
        if word in pos_words_en:
            score+= 1
        if word in neg_words_en:
            score-=1
        return id, score
    elif lang == "es":
        if word in pos_words_es:
            score+= 1
        if word in neg_words_es:
            score-=1
        return id, score
    elif lang == "fr":
        if word in pos_words_fr:
            score+= 1
        if word in neg_words_fr:
            score-=1
        return id, score
    elif lang == "tr":
        if word in pos_words_tr:
            score+= 1
        if word in neg_words_tr:
            score-=1
        return id, score
    else: #italian
        if word in pos_words_it:
            score+= 1
        if word in neg_words_it:
            score-=1
        return id, score
    return id, score

In [10]:
%%time
#removing header
twitter_rdd = sc.textFile("twitter_full_cleaned/twitter_full.csv")
header=twitter_rdd.first()
twitter_rdd=twitter_rdd.filter(lambda row : row!= header)

CPU times: user 3.01 ms, sys: 2.83 ms, total: 5.84 ms
Wall time: 1.33 s


In [11]:
%%time
#MapReduce
twitter_id_word_pair_rdd=twitter_rdd.flatMap(twitter_id_word_pair)
twitter_sentiment_pair_rdd=twitter_id_word_pair_rdd.map(twitter_word_to_sentiment)
twitter_comment_scores=twitter_sentiment_pair_rdd.reduceByKey(lambda x,y: x+y)

CPU times: user 18.7 ms, sys: 6.21 ms, total: 24.9 ms
Wall time: 66.3 ms


In [12]:
%%time
twitter_positive_scores=twitter_comment_scores.filter(lambda x: x[1]>0)
twitter_neutral_scores=twitter_comment_scores.filter(lambda x: x[1]==0)
twitter_negative_scores=twitter_comment_scores.filter(lambda x: x[1]<0)

CPU times: user 37 μs, sys: 1e+03 ns, total: 38 μs
Wall time: 41.2 μs


In [13]:
%%time
total=twitter_comment_scores.count()
print(f"total: {total}")
twitter_positive_percent=twitter_positive_scores.count()/total
twitter_neutral_percent=twitter_neutral_scores.count()/total
twitter_negative_percent=twitter_negative_scores.count()/total
print(f"pos: {twitter_positive_percent} neutral: {twitter_neutral_percent} neg: {twitter_negative_percent}")

total: 7473158


[Stage 8:====================================>                    (32 + 8) / 50]

pos: 0.27621923690091926 neutral: 0.3765329195502089 neg: 0.34724784354887184
CPU times: user 59.2 ms, sys: 44.8 ms, total: 104 ms
Wall time: 42.2 s
